# 2.  Build a KNN model with your selected stock / market index using your  best  distance  measure,  determine  the  number  attributes that is capable of giving the best prediction.  (Select attributesin ascending order(ie 3, 5, 7, ...) and determine the accuracy forthe selected attributes, compare the accuracies to find which isthe best)

In [187]:
import pandas as pd
import os


DATA_FILEPATH = "data/CAC40.csv"

def fetch_data():
    """Import the data from csv to pd dataframe"""
    relaviteFilepath = os.path.join(os.path.abspath(''), DATA_FILEPATH)
    return pd.read_csv(relaviteFilepath)

# df stands for dataframe. This is the object that we will manipulate throughouht the notebook
cac40df = fetch_data()
cac40df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1990-03-01,1836.000000,1838.000000,1827.000000,1832.000000,1832.000000,0
1,1990-03-02,1831.000000,1860.000000,1831.000000,1860.000000,1860.000000,0
2,1990-03-05,1866.000000,1874.000000,1862.000000,1874.000000,1874.000000,0
3,1990-03-06,1869.000000,1875.000000,1866.000000,1872.000000,1872.000000,0
4,1990-03-07,1874.000000,1881.000000,1874.000000,1880.000000,1880.000000,0
...,...,...,...,...,...,...,...
7999,2021-09-15,6654.830078,6659.270020,6577.020020,6583.620117,6583.620117,81904200
8000,2021-09-16,6613.709961,6663.410156,6612.160156,6622.589844,6622.589844,79574500
8001,2021-09-17,6679.450195,6697.080078,6551.620117,6570.189941,6570.189941,214025500
8002,2021-09-20,6450.390137,6471.089844,6389.620117,6455.810059,6455.810059,120069400


# Preprocessing
We can remove the date col as it is useless for predictions

In [188]:
cac40df.drop(columns=['Date'], axis=1, inplace=True)

We add some features and the labels

In [189]:
maWindows = [20, 50, 200]


for maWindow in maWindows:
    colName = f"ma{maWindow}"
    cac40df[colName] = cac40df["Close"].rolling(maWindow).mean()



cac40df

,Open,High,Low,Close,Adj Close,Volume,ma20,ma50,ma200
0,1836.000000,1838.000000,1827.000000,1832.000000,1832.000000,0,NaN,NaN,NaN
1,1831.000000,1860.000000,1831.000000,1860.000000,1860.000000,0,NaN,NaN,NaN
2,1866.000000,1874.000000,1862.000000,1874.000000,1874.000000,0,NaN,NaN,NaN
3,1869.000000,1875.000000,1866.000000,1872.000000,1872.000000,0,NaN,NaN,NaN
4,1874.000000,1881.000000,1874.000000,1880.000000,1880.000000,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7999,6654.830078,6659.270020,6577.020020,6583.620117,6583.620117,81904200,6679.177539,6654.120215,6180.638843
8000,6613.709961,6663.410156,6612.160156,6622.589844,6622.589844,79574500,6680.012524,6658.637412,6185.706042
8001,6679.450195,6697.080078,6551.620117,6570.189941,6570.189941,214025500,6677.216528,6659.452812,6190.690093
8002,6450.390137,6471.089844,6389.620117,6455.810059,6455.810059,120069400,6665.852026,6657.384014,6195.165793


In [190]:
from math import isnan

def prediction(number):
    #Up
    if isnan(number):
        return float('NaN')
    if number > threshold:
        return 1 
    #Down
    if number < -threshold:
        return 2
    #Flat
    return 0

In [191]:
#Daily return on closing price
cac40df["Daily_return"] = (cac40df["Close"] - cac40df["Close"].shift(1)) / cac40df["Close"].shift(1)

#Label
threshold = 0.005
cac40df["Prediction_nbr"] = cac40df["Daily_return"].apply(prediction)
print(cac40df)
#Shift to have next day prediction
cac40df["Prediction_nbr"] = cac40df["Prediction_nbr"].shift(-1)

#Drop all the cols with Nana (MA, shifts)
cac40df.dropna(inplace=True)
cac40df

             Open         High          Low        Close    Adj Close  \
0     1836.000000  1838.000000  1827.000000  1832.000000  1832.000000   
1     1831.000000  1860.000000  1831.000000  1860.000000  1860.000000   
2     1866.000000  1874.000000  1862.000000  1874.000000  1874.000000   
3     1869.000000  1875.000000  1866.000000  1872.000000  1872.000000   
4     1874.000000  1881.000000  1874.000000  1880.000000  1880.000000   
...           ...          ...          ...          ...          ...   
7999  6654.830078  6659.270020  6577.020020  6583.620117  6583.620117   
8000  6613.709961  6663.410156  6612.160156  6622.589844  6622.589844   
8001  6679.450195  6697.080078  6551.620117  6570.189941  6570.189941   
8002  6450.390137  6471.089844  6389.620117  6455.810059  6455.810059   
8003  6513.750000  6570.129883  6513.750000  6552.729980  6552.729980   

         Volume         ma20         ma50        ma200  Daily_return  \
0             0          NaN          NaN          

,Open,High,Low,Close,Adj Close,Volume,ma20,ma50,ma200,Daily_return,Prediction_nbr
199,1602.000000,1613.000000,1586.000000,1591.000000,1591.000000,0,1638.300000,1621.900000,1833.600000,-0.009340,0.0
200,1619.000000,1619.000000,1578.000000,1589.000000,1589.000000,0,1636.400000,1622.660000,1832.385000,-0.001257,2.0
201,1574.000000,1576.000000,1541.000000,1553.000000,1553.000000,0,1632.100000,1622.920000,1830.850000,-0.022656,1.0
202,1542.000000,1572.000000,1539.000000,1564.000000,1564.000000,0,1628.300000,1623.360000,1829.300000,0.007083,2.0
203,1553.000000,1572.000000,1537.000000,1537.000000,1537.000000,0,1624.800000,1622.600000,1827.625000,-0.017263,2.0
...,...,...,...,...,...,...,...,...,...,...,...
7998,6672.189941,6677.069824,6613.520020,6652.970215,6652.970215,63626300,6688.502026,6653.002217,6175.592542,-0.003588,2.0
7999,6654.830078,6659.270020,6577.020020,6583.620117,6583.620117,81904200,6679.177539,6654.120215,6180.638843,-0.010424,1.0
8000,6613.709961,6663.410156,6612.160156,6622.589844,6622.589844,79574500,6680.012524,6658.637412,6185.706042,0.005919,2.0
8001,6679.450195,6697.080078,6551.620117,6570.189941,6570.189941,214025500,6677.216528,6659.452812,6190.690093,-0.007912,2.0


In [192]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [193]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

maxAcc, maxDist, maxAttr = 0, "", 0

for attributeNumber in range(1, len(cac40df.columns)):
    #Take only i cols
    cac40Truncated = cac40df.iloc[: , :attributeNumber]
    try:
        cac40Truncated.drop(columns=['Prediction_nbr'], axis=1, inplace=False)
    except:
        pass

    #Build test, train
    X,y = cac40Truncated, cac40df["Prediction_nbr"]
    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.75, random_state=42, stratify=y)

    #Test every distance
    currentMeanAcc = 0
    for dist in ["manhattan", "euclidean", "cosine"]:
        kClf = KNeighborsClassifier(metric=dist)
        kClf.fit(X_train, y_train)
        y_pred = kClf.predict(X_test)
        currAcc = round(100*accuracy_score(y_test, y_pred), 1)

        currentMeanAcc += currAcc
        
        if currAcc > maxAcc:
            maxAcc, maxDist, maxAttr = currAcc, dist, attributeNumber
    print(f"{attributeNumber} attributes yielded a mean of {currentMeanAcc/3}% accuracy")

print(f"Distance {maxDist} yielded the maximum accuracy of {maxAcc}% with {maxAttr} attributes")

1 attributes yielded a mean of 36.766666666666666% accuracy
2 attributes yielded a mean of 36.43333333333334% accuracy
3 attributes yielded a mean of 38.1% accuracy
4 attributes yielded a mean of 38.166666666666664% accuracy
5 attributes yielded a mean of 38.23333333333333% accuracy
6 attributes yielded a mean of 37.73333333333333% accuracy
7 attributes yielded a mean of 38.233333333333334% accuracy
8 attributes yielded a mean of 38.93333333333333% accuracy
9 attributes yielded a mean of 38.833333333333336% accuracy
10 attributes yielded a mean of 38.666666666666664% accuracy
Distance manhattan yielded the maximum accuracy of 39.3% with 10 attributes


The accuracy does not strictly go up, but is still pretty decent with a good number of attributes. It is to be noted that the number of features used is not the only factor - the relevance of the features is way more important (ex: Having ma20, ma50, ma100 ,etc... without any of the oscillators or other advanced features will be pejorative). Contrarly to the Bayes, the dependance of the features does not seem to be harmfull in any way.\
\
The 38% accuracy is first and foremost very bad (worst than a random decision of 33% with a ternary decision) and is achieved at 3 attributes, the next steps bringing almost no amelioration. It could be interesting to know if any of the advanced features (stocastic oscillator, signals) would yield better results with fewer attributes. 